In [1]:

from tensorflow.keras.models import load_model
model = load_model('portuguese_chatbot.h5')

2025-04-01 00:30:11.188227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743478211.202721  226145 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743478211.206768  226145 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743478211.219035  226145 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743478211.219061  226145 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743478211.219062  226145 computation_placer.cc:177] computation placer alr

In [2]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ (None, 25, 128)        │        47,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 25, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_26                │ (None, 25, 512)        │       788,480 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 25, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_27                │ (None, 25, 512)        │     1,574,912 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 25, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 25, 370)        │       189,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,605,172 (9.94 MB)

 Trainable params: 2,602,866 (9.93 MB)

 Non-trainable params: 2,304 (9.00 KB)

 Optimizer params: 2 (12.00 B)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 




In [18]:
from tensorflow.keras.models import load_model
import pickle
import unicodedata
import re
import nltk
# nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


model = load_model('portuguese_chatbot.h5')
max_length = 100

# Load tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

def preprocess_text(text):
    # Normalize and clean text
    text = unicodedata.normalize('NFKD', text.lower()).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^\w\s]', '', text)
    return text

def generate_response(input_text, temperature=0.7):
    # Preprocess input
    cleaned_input = preprocess_text(input_text)
    input_seq = tokenizer.texts_to_sequences([cleaned_input])
    input_padded = pad_sequences(input_seq, maxlen=max_length, padding='post')
    
    # Initialize response
    response_seq = np.zeros((1, max_length))
    response_seq[0, 0] = tokenizer.word_index['<OOV>']  # Start token
    
    for i in range(1, max_length):
        # Predict next word
        predictions = model.predict([input_padded, response_seq], verbose=0)[0][i-1]
        
        # Apply temperature for diversity
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        
        # Sample from predictions
        next_word_idx = np.random.choice(len(predictions), p=predictions)
        response_seq[0, i] = next_word_idx
        
        # Stop if end token is predicted
        if next_word_idx == 0:
            break
    
    # Convert sequence to text
    response_tokens = [tokenizer.index_word.get(idx, '') for idx in response_seq[0] if idx != 0]
    return ' '.join(response_tokens)

# Test the chatbot
test_inputs = [
    "oi",
    "conte-me sobre o folclore",
    "quais frutas tem na amazônia?",
    "quem é o curupira2?"
]

for input_text in test_inputs:
    response = generate_response(input_text)
    print(f"User: {input_text}")
    print(f"Chatbot: {response}\n")

/home/prinako/External/School/UFPA/ITEC_UFPA/24.4_SEMESTER/INTELIGENCIA_COMPUTACIONAL/T2/amazon-portuguese-chatbot/venv/lib/python3.11/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 100))', 'Tensor(shape=(1, 100))')
  warnings.warn(msg)


User: oi
Chatbot: <OOV> para de quer de festa 10 da fica belos prato voce pobreza agosto recomendo 2 tipicas destes umido revitalizado hidrovia floresta periodos porto tem chegar

User: conte-me sobre o folclore
Chatbot: <OOV> por gostaria borracha 2000 maio tem festa catolica oncas mais origem fauna 1 dezembro marujada vivem saber saber oi virados arraial rios sopa campos boibumba internacional norte origem museu docas junho saber presepio 5 mae conhecida jari belemjulio ola belemjulio estacao guama sopa catolica tipicas 26c incriveis temperaturas lingua umido tapajos rua campos cabelos recomendo ouvir dia teve noite conhecida na destes fluvial conteme fundada significa campos cabanagem culinaria ola cultural reconhecida contra ou umido 10 norte imperdiveis lenda

User: quais frutas tem na amazônia?
Chatbot: <OOV> mil temperaturas cabanagem cabelos bom complexo em como sobre 26c cozido xingu aereo recomendo indigenas macacos eventos regiao antigo camarao 10 ou tapajos museu famoso cli

In [16]:
def generate_response(input_text, temperature=0.8):
    """
    Generate a response to the input text using the trained model.
    
    Args:
        input_text: User input string
        temperature: Controls randomness of predictions (higher = more random)
        
    Returns:
        Generated response string
    """
    # Preprocess input
    cleaned_input = preprocess_text(input_text)
    input_seq = tokenizer.texts_to_sequences([cleaned_input])
    input_padded = pad_sequences(input_seq, maxlen=max_length, padding='post')
    
    # Initialize response
    response_seq = np.zeros((1, max_length))
    response_seq[0, 0] = tokenizer.word_index['<OOV>']  # Start token
    
    for i in range(1, max_length):
        # Predict next word
        predictions = model.predict([input_padded, response_seq], verbose=0)[0][i-1]
        
        # Apply temperature for diversity
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        
        # Sample from predictions
        next_word_idx = np.random.choice(len(predictions), p=predictions)
        response_seq[0, i] = next_word_idx
        
        # Stop if end token is predicted
        if next_word_idx == 0:
            break
    
    # Convert sequence to text
    response_tokens = []
    for idx in response_seq[0]:
        if idx == 0:  # Skip padding
            continue
        word = tokenizer.index_word.get(idx, '')
        if word == '<OOV>':  # Skip OOV tokens
            continue
        response_tokens.append(word)
    
    return ' '.join(response_tokens)

# Test the chatbot
test_inputs = [
    "oi",
    "conte-me sobre o folclore",
    "quais frutas tem na amazônia?",
    "quem é o curupira?",
    "o que é Ver-o-Peso"
]

for input_text in test_inputs:
    response = generate_response(input_text)
    print(f"User: {input_text}")
    print(f"Chatbot: {response}\n")

User: oi
Chatbot: conte boa nazaré região paz dias açaí margens falar aeroporto origem estou lenda destacamse tradições castelo goeldi. chamada mandioca cezar explosão nazaré

User: conte-me sobre o folclore
Chatbot: mangal posso nazaré. teatro hoje melhor cabelos garças a o durante de tudo da palacetes tudo hoje essencial carnes.

User: quais frutas tem na amazônia?
Chatbot: km² 1 sopa mistura construção estado fluvial a o destacamse o a revitalizado que que conhecer morte você rodoviário falar

User: quem é o curupira?
Chatbot: portuguesas africanas região. história das docas. feita jambu você o internacional 3 catirina catirina seco aeroporto floresta. contar. ressurreição teatro código grãopará do belos nazaré?

User: o que é Ver-o-Peso
Chatbot: na clima nazaré km protege o 3 oferece foi nazaré. o conversar 1878 afroindígena novembro maior

